In [1]:
!pip install yfpy

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created 


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd

In [ ]:
from yfpy.query import YahooFantasySportsQuery

query = YahooFantasySportsQuery(
    league_id="67269",
    game_code="nhl",
    game_id=453,
    yahoo_consumer_key="",
    yahoo_consumer_secret=""
)

In [48]:
LEAGUE_KEY = query.get_league_key(season=None)

In [3]:
TEAMS = query.get_league_teams()

In [14]:
LEAGUE_SETTINGS = query.get_league_settings()
SCORING_CATEGORIES = LEAGUE_SETTINGS.stat_categories.stats
for i in range(len(SCORING_CATEGORIES)):
    SCORING_CATEGORIES[i] = SCORING_CATEGORIES[i].clean_data_dict()
SCORING_CATEGORIES = pd.DataFrame(data=SCORING_CATEGORIES)

SCORING_MODIFIERS = data=LEAGUE_SETTINGS.stat_modifiers.stats
for i in range(len(SCORING_MODIFIERS)):
    SCORING_MODIFIERS[i] = SCORING_MODIFIERS[i].clean_data_dict()
SCORING_MODIFIERS = pd.DataFrame(data=SCORING_MODIFIERS)

SCORING_SETTINGS = pd.merge(left=SCORING_CATEGORIES, right=SCORING_MODIFIERS, on='stat_id', how='inner')

Draft Info

In [21]:
draft = query.get_league_draft_results()

Seperate Draft into by Team

In [49]:
draft = pd.DataFrame(data=draft, columns=['Overall', 'Round', 'Team ID', 'Player ID'])

team_drafts = {}
team_keys = []
for ID in range(1, draft['Team ID'].nunique()+1):
    team_keys = [LEAGUE_KEY + '.t.' + str(ID) for ID in range(1, draft['Team ID'].nunique()+1)] 
team_keys = tuple(team_keys)

In [ ]:
team_drafts = {team_key : draft[draft['Team ID'] == team_key] for team_key in team_keys}

Total Points for each Team

In [46]:
mcdavid_stats = query.get_player_stats_for_season(player_key="453.p.6743").player_stats
print(mcdavid_stats)
total_draft_points = {}

PlayerStats({
  "coverage_type": "season",
  "season": 2024,
  "stats": [
    {
      "stat": {
        "stat_id": 1,
        "value": 22.0
      }
    },
    {
      "stat": {
        "stat_id": 2,
        "value": 49.0
      }
    },
    {
      "stat": {
        "stat_id": 4,
        "value": 10.0
      }
    },
    {
      "stat": {
        "stat_id": 8,
        "value": 23.0
      }
    },
    {
      "stat": {
        "stat_id": 11,
        "value": 0.0
      }
    },
    {
      "stat": {
        "stat_id": 14,
        "value": 155.0
      }
    },
    {
      "stat": {
        "stat_id": 31,
        "value": 29.0
      }
    },
    {
      "stat": {
        "stat_id": 32,
        "value": 9.0
      }
    }
  ]
})


Assign Best and Worst Draft

Playground

In [12]:
Mcdavid = query.get_player_stats_for_season("453.p.6743")

In [13]:
Mcdavid.stats

[Stat({
   "stat_id": 1,
   "value": 22.0
 }),
 Stat({
   "stat_id": 2,
   "value": 49.0
 }),
 Stat({
   "stat_id": 4,
   "value": 10.0
 }),
 Stat({
   "stat_id": 8,
   "value": 23.0
 }),
 Stat({
   "stat_id": 11,
   "value": 0.0
 }),
 Stat({
   "stat_id": 14,
   "value": 155.0
 }),
 Stat({
   "stat_id": 31,
   "value": 29.0
 }),
 Stat({
   "stat_id": 32,
   "value": 9.0
 })]

In [15]:
query.get_league_matchups_by_week(18)

[Matchup({
   "is_consolation": 0,
   "is_playoffs": 0,
   "status": "preevent",
   "teams": [
     {
       "team": {
         "draft_position": 1,
         "has_draft_grade": 0,
         "is_owned_by_current_login": 1,
         "league_scoring_type": "headpoint",
         "managers": {
           "manager": {
             "email": "amtadam14@hotmail.ca",
             "felo_score": 571,
             "felo_tier": "bronze",
             "guid": "NGSXRXJK2U3UJVTCC2PO3L2A6M",
             "image_url": "https://s.yimg.com/ag/images/default_user_profile_pic_64sq.jpg",
             "is_commissioner": 0,
             "is_current_login": 1,
             "manager_id": 1,
             "nickname": "adam"
           }
         },
         "name": "McFuck",
         "number_of_moves": 38,
         "number_of_trades": 0,
         "roster_adds": {
           "coverage_type": "week",
           "coverage_value": 17,
           "value": 0
         },
         "team_id": 1,
         "team_key": "453.l.6

In [14]:
transactions = query.get_league_transactions()

In [16]:
adds = {}
for transaction in transactions:
    if transaction.type == "drop":
        print(transaction.players)

[Player({
  "display_position": "D",
  "editorial_team_abbr": "CBJ",
  "name": {
    "ascii_first": "Ivan",
    "ascii_last": "Provorov",
    "first": "Ivan",
    "full": "Ivan Provorov",
    "last": "Provorov"
  },
  "player_id": 6749,
  "player_key": "453.p.6749",
  "position_type": "P",
  "transaction_data": {
    "destination_type": "waivers",
    "source_team_key": "453.l.67269.t.4",
    "source_team_name": "(",
    "source_type": "team",
    "type": "drop"
  }
})]
[Player({
  "display_position": "C,RW",
  "editorial_team_abbr": "UTA",
  "name": {
    "ascii_first": "Nick",
    "ascii_last": "Schmaltz",
    "first": "Nick",
    "full": "Nick Schmaltz",
    "last": "Schmaltz"
  },
  "player_id": 6386,
  "player_key": "453.p.6386",
  "position_type": "P",
  "transaction_data": {
    "destination_type": "waivers",
    "source_team_key": "453.l.67269.t.4",
    "source_team_name": "(",
    "source_type": "team",
    "type": "drop"
  }
})]
[Player({
  "display_position": "C",
  "editori

In [18]:
print(transactions[1])

Transaction({
  "players": {
    "player": {
      "display_position": "C,RW",
      "editorial_team_abbr": "UTA",
      "name": {
        "ascii_first": "Nick",
        "ascii_last": "Schmaltz",
        "first": "Nick",
        "full": "Nick Schmaltz",
        "last": "Schmaltz"
      },
      "player_id": 6386,
      "player_key": "453.p.6386",
      "position_type": "P",
      "transaction_data": {
        "destination_type": "waivers",
        "source_team_key": "453.l.67269.t.4",
        "source_team_name": "(",
        "source_type": "team",
        "type": "drop"
      }
    }
  },
  "status": "successful",
  "timestamp": 1739232690,
  "transaction_id": 525,
  "transaction_key": "453.l.67269.tr.525",
  "type": "drop"
})
